# Modelling autocorrelation in the sea level

The statistical tests and model diagnostics show autocorrelation in the sea levels. This notebook tests the AR(1) specification in the model with storm surge and implicitly adds the flexibility of one or more structural breaks in the trend parameter. 

Note that we need the package PyFlux as the AR(1) model with explanatory variables is in the class of ARIMAX models. The package StatsModels cannot deal with explanatory variables. 

In [1]:
# Standard Python packages
import io

# Python packages that need to be installed using pip or anaconda:
# For computations
import pandas
import numpy as np

# For plotting
import matplotlib.pyplot as plt
import bokeh.palettes
import bokeh.plotting
from bokeh.models import HoverTool

# Initialize modules for the jupyter notebook format
from nbformat import current
%matplotlib inline
bokeh.io.output_notebook()

from IPython.display import display
import shutil

import statsmodels.graphics.regressionplots as plots

# Disable pandas warnings
pandas.options.mode.chained_assignment = None

C:\Users\nicolai.HKV\AppData\Local\Continuum\anaconda3\lib\site-packages\nbformat\current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


Loading BokehJS ...

In [2]:
def execute_notebook(nbfile):
    """Function to run other notebook in this notebook"""
    with io.open(nbfile,encoding="utf8") as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

In [3]:
# Load notebook with basic io functionality (wind, PSMSL) and standard linear model
execute_notebook('../satellite/get-data.ipynb')

In [4]:
# laad eerste het model met de summary
station_names = [
    'Vlissingen', 
    'Hoek van Holland', 
    'Den Helder', 
    'Delfzijl', 
    'Harlingen', 
    'IJmuiden'
]

# Locatie wind data is 50 km uit kust vanuit IJmuiden
rlr_data = get_station_data(dataset_name='rlr_annual', coastline_code=150, names=station_names, include_wind=False)

rlr_annual 20 rlr
rlr_annual 22 rlr
rlr_annual 23 rlr
rlr_annual 24 rlr
rlr_annual 25 rlr
rlr_annual 32 rlr


In [5]:
stations = [20, 22, 23, 24, 25, 32]

grouped = pandas.concat(rlr_data.loc[stations, 'data'].tolist())[['year', 'height']].groupby(['year'])
mean_df = grouped.mean().reset_index()
# filter out non-trusted part (before NAP)
mean_df = mean_df[mean_df['year'] >= 1890]

station_names = [st.capitalize() for st in rlr_data.loc[stations, 'name'].tolist()]

print(f'The sea water level data of the following stations are analyzed: {", ".join(station_names)}')

The sea water level data of the following stations are analyzed: Vlissingen, Hoek van holland, Den helder, Delfzijl, Harlingen, Ijmuiden


In [6]:
mean_df.index = mean_df['year']
display(mean_df.head(), mean_df.tail())

,year,height
year,,
1890,1890,-194.666667
1891,1891,-179.000000
1892,1892,-166.500000
1893,1893,-142.166667
1894,1894,-141.833333


,year,height
year,,
2013,2013,25.666667
2014,2014,66.666667
2015,2015,92.500000
2016,2016,77.000000
2017,2017,112.166667


In [7]:
# Sea level data range
print(f'Sea level data for the period')
print(min(mean_df.year.values), max(mean_df.year.values))

Sea level data for the period
1890 2017


In [8]:
import getpass
username = getpass.getuser()
print(username)

if username == 'rongen': 
    surgepath = 'D:/Documents/2695.50 Zeespiegelstijging 2018/Data/'
elif username == 'nicolai':
    surgepath = 'D:/Users/' + username + '/Documents/2695.50 Zeespiegelstijging 2018/Data/'
    print(surgepath)
else:
#    print('enter path where surge is stored using forward slashes')
#    surgepath = input()
#    print(surgepath)
    print('path not valid')

#print(surgepath + 'surge.pkl')   

#'D:\Users\Nicolai\Documents\2695.50 Zeespiegelstijging 2018\Data'

# Add surge
# Load surge and convert from meters to mm
#surge = pandas.read_pickle(surgepath + 'surge.pkl') * 1000
#print(surge.head(5), surge.tail(5))

nicolai
D:/Users/nicolai/Documents/2695.50 Zeespiegelstijging 2018/Data/


In [9]:
# Add surge
# Load surge and convert from meters to mm

# Guus
# surge = pandas.read_pickle('D:/Documents/2695.50 Zeespiegelstijging 2018/Data/surge.pkl') * 1000 

# Robin
surgepath = 'D:/Users/Nicolai/Documents/2695.50 Zeespiegelstijging 2018/Data/'
surge = pandas.read_pickle(surgepath + 'surge.pkl') * 1000

print(surge.head(5), surge.tail(5))

# Calculate average over given stations, per year
# if monthly averages are needed, groupby "surge.index.month"
average = surge[station_names].groupby(surge.index.year).mean().mean(axis=1)
# Construct dataframe and add to mean_df
surge_per_year = pandas.DataFrame(data=[average.mean()] * len(mean_df), index=mean_df['year'], columns=['surge'])
surge_per_year.loc[average.index, 'surge'] = average.values
surge_per_year.index.name = 'year'
if 'surge' not in mean_df.columns:
    mean_df = mean_df.merge(surge_per_year.reset_index(), on='year')

# Create a corrected dataframe by subtracting the surge
mean_df_corrected = mean_df.copy()
mean_df_corrected['height'] -= mean_df_corrected['surge']

                     Vlissingen  Hoek van holland  Den helder   Delfzijl  \
1979-01-01 00:00:00    0.000000          0.000000    0.000000   0.000000   
1979-01-01 00:10:00   -0.249468          0.590325    0.806396  -3.677294   
1979-01-01 00:20:00   -0.890010          1.760389    3.596331 -11.441492   
1979-01-01 00:30:00   -1.303061          2.307753    7.541155 -16.544901   
1979-01-01 00:40:00   -1.140522          1.127529   10.611323 -14.176514   

                     Harlingen  Ijmuiden  
1979-01-01 00:00:00   0.000000  0.000000  
1979-01-01 00:10:00  -2.944402 -0.753033  
1979-01-01 00:20:00  -5.254048 -2.288197  
1979-01-01 00:30:00  -4.505036 -3.556560  
1979-01-01 00:40:00  -5.693127 -4.844942                        Vlissingen  Hoek van holland  Den helder   Delfzijl  \
2014-12-31 23:10:00 -177.668601       -153.670296  -95.220782  15.352673   
2014-12-31 23:20:00 -181.090802       -151.775345  -74.545339  10.494649   
2014-12-31 23:30:00 -181.896344       -149.892956  -74.13

C:\Users\nicolai.HKV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: 'year' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


In [10]:
# storm surge data range

print(f'Storm surge data for the period')
print(min(surge.index.year), '-', max(surge.index.year))

Storm surge data for the period
1979 - 2014


Run the linear regression model with storm surge (no robust standard errors).

In [11]:
def linear_model_with_surge(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613),
        df['surge']# * (df['year'] >= 1979)    
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']
        
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    fit = model.fit()
    #fit = model.fit(cov_type='HC0')
    #fit = model.fit(cov_type='HAC')
    return fit, names

In [12]:
linear_fit_with_surge, names = linear_model_with_surge(mean_df)
table = linear_fit_with_surge.summary(
    yname='Sea-surface height', 
    xname=names, 
    title='Linear model (1890 - current), with surge (1979-current)'
)
display(table)

<class 'statsmodels.iolib.summary.Summary'>
"""
           Linear model (1890 - current), with surge (1979-current)           
==============================================================================
Dep. Variable:     Sea-surface height   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     255.2
Date:                Fri, 14 Sep 2018   Prob (F-statistic):           1.54e-58
Time:                        19:16:39   Log-Likelihood:                -593.96
No. Observations:                 128   AIC:                             1198.
Df Residuals:                     123   BIC:                             1212.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Constant     -23.1142      2.482     -9.313      0.000     -28.027     -18.201
Trend          1.9267      0.062     31.324      0.000       1.805       2.048
Nodal U        4.5272      3.210      1.410      0.161      -1.827      10.881
Nodal V      -10.8684      3.212     -3.384      0.001     -17.226      -4.511
Surge          0.9018      0.144      6.259      0.000       0.617       1.187
==============================================================================
Omnibus:                        3.042   Durbin-Watson:                   1.505
Prob(Omnibus):                  0.219   Jarque-Bera (JB):                2.517
Skew:                          -0.258   Prob(JB):                        0.284
Kurtosis:                       3.454   Cond. No.                         57.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The Durbin-Watson test, Breusch-Godfrey test, and (partial) autocorrelation function show that this model exhibits autocorrelation with lag 1 [see notebook Statistical Tests]. Other literature on sea level regression finds this behaviour as well. Furthermore, the residuals are heteroskedastic, i.e. they depend on the value of the explanatory variables. One way to deal with both issues is estimating robust standard errors. The HAC estimator takes into account both heteroskedasticity and autocorrelation [see notebook Statistical Tests]. Below we shall introduce an AR(1) model to capture the autocorrelation in the signal.

In [13]:
import scipy.stats as stats
import statsmodels.tsa.ar_model as ar_model
import statsmodels.tsa.stattools as ar_tools

#mean_df.head()
ar = ar_model.AR(mean_df['height'])
#print(ar.information(model_ar.params))

model_ar = ar.fit(method='mle', maxlag=1, trend='ct')
print(model_ar.params)

ar_results = ar_model.ARResults(ar, model_ar.params)

print("")
print("confidence interval")
print(ar_results.conf_int())

print("")
print("standard errors")
print(ar_results.bse)

print("")
print("t-values")
print(ar_results.tvalues)
      
ar_tools.adfuller(mean_df['height'], maxlag = 1, regression='ct')

const       -135.640573
trend          1.467085
L1.height      0.230480
dtype: float64

confidence interval
[[-1.67319629e+02 -1.03961518e+02]
 [ 1.12058877e+00  1.81358144e+00]
 [ 6.07140011e-02  4.00245960e-01]]

standard errors
[16.16308042  0.1767871   0.08661689]

t-values
const       -8.392000
trend        8.298598
L1.height    2.660913
dtype: float64


(-7.932728955793214,
 1.0222059792980013e-10,
 1,
 126,
 {'1%': -4.032477692253856,
  '5%': -3.4459262881778225,
  '10%': -3.1478290513383484},
 1216.7811025960364)

The AR(1) model with constant and trend (without minus 1970) leads to a smaller trend. 

The AR(1) parameter does not seem to be equal to 1. The null hypothesis of the Augmented Dickey-Fuller test, there is a unit root, is rejected. 

Now we will add the other variables to the equation.

In [14]:
import pyflux as pf

In [15]:
def linear_AR_model_with_surge(df):
    """
    Return the fit from the (linear) ARX(1) model on the given dataset df.
    """
    df2 = df.copy()
    df2['height'] = df2['height']
    df2['nodalcos'] = np.cos(2*np.pi*(df2['year']-1970)/18.613)
    df2['nodalsin'] = np.sin(2*np.pi*(df2['year']-1970)/18.613)
    df2['trend'] = (df2['year']-1970)
    df2['surge'] = df2['surge']
    
    print(df2.head())
    
    names = ['AR(1)', 'Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']   
    
    model = pf.ARIMAX(data=df2, formula='height~1+trend+nodalcos+nodalsin+surge',
                  ar=1, ma=0, integ=0,family=pf.Normal())
  
    fit = model.fit("MLE",cov_type='HC0')
    #fit = model.fit("MLE")
    return model, fit, names

In [16]:
print(mean_df.head())
ar_model, ar_fit_with_surge, names = linear_AR_model_with_surge(mean_df)
ar_fit_with_surge.summary()

#table = ar_fit_with_surge.summary(
#    yname='Sea-surface height', 
#    xname=names, 
#    title='Linear AR(1) model (1890 - current), with surge (1979-current)'
#)
#display(table)
print(mean_df.head()) # Why does mean_df change?

   year      height     surge
0  1890 -194.666667 -0.960549
1  1891 -179.000000 -0.960549
2  1892 -166.500000 -0.960549
3  1893 -142.166667 -0.960549
4  1894 -141.833333 -0.960549
   year      height     surge  nodalcos  nodalsin  trend
0  1890 -194.666667 -0.960549 -0.297469 -0.954731    -80
1  1891 -179.000000 -0.960549  0.035522 -0.999369    -79
2  1892 -166.500000 -0.960549  0.364503 -0.931202    -78
3  1893 -142.166667 -0.960549  0.652341 -0.757926    -77
4  1894 -141.833333 -0.960549  0.866545 -0.499098    -76
Normal ARIMAX(1,0,0)                                                                                      
======================================================= ==================================================
Dependent Variable: height                              Method: MLE                                       
Start Date: 1                                           Log Likelihood: -587.3706                         
End Date: 127                                     

fit = model.fit('MLE')


fit = model.fit('MLE',covtype='HC0')


The ARX(1) model fits the model quite well. The trend parameter is smaller than it was in the linear model.

In [ ]:
ar_model.plot_fit(figsize=(15,10))
ar_model.plot_predict(h=10, oos_data=mean_df.iloc[-12:], past_values=100, figsize=(15,5))

# Model diagnostics : to do for the ARX(1) model
## Normality
Let us first check if the (fitted) residuals are normally distributed. We do several visual checks and consider statistical tests.

### Statistical test
The Jarque-Bera test statistic is 2.517 with p-value 0.284. Hence, the null hypothesis of normality cannot be rejected at a significance level of 5%. [To do: including numbers from table in text.]

### Q-Q-plot of the residuals
Next we create the quantile-quantile plot of the residuals. That is, we plot the theoretical quantiles (normal distribution) against the data quantiles (ranks).

The first plot relates to the residuals of the full model, the second plot to the residuals from 1979 from the full model and the third plot relates to the residuals of the model fitted to the data from 1979 onwards.

The Q-Q-plots do not show large irregularities in the residuals. Higher quantiles (0.7-0.9) are somewhat overrepresented and the quantiles 0.3-0.5 are somewhat underrepresented (first plot, full model). 

In [ ]:
fig, axs = plt.subplots(figsize=(14, 4), ncols=3)
from scipy.stats import norm

for startyear, ax, fit, title in zip(
    [1890, 1979, 1979],
    axs,
    [linear_fit_with_surge, linear_fit_with_surge, linear_fit_with_surge_1979],
    ['All residuals from fit 1890-2016', 'Residuals 1979-2016 from fit 1890-2016', 'Residuals from fit 1979-2016']
):
    
    years = fit.model.exog[:, 1] + 1970
    index = (years >= startyear)
#     print(index)

    
    ax.set_aspect(1.0)

    # Calculate the exceedance probabilities given a normal distribution
    residuals = fit.resid.values[index]
    cprob = norm.cdf(
        sorted(residuals),
        loc=residuals.mean(),
        scale=residuals.std()
    )
    # Calculate the data quantiles: rank / (n + 1)
#     print(fit.resid.rank().values)
#     print(np.argsort(residuals))
    ranks = (np.arange(len(residuals)) + 1) / (len(residuals) + 1)

    # Scatter
    ax.plot(cprob, ranks, marker='.', ls='')
    # Plot the 1-1 line
    ax.plot([0, 1], [0, 1], ls='-', color='0.5')

    # Layout
    ax.grid()
    ax.set_ylabel('Normal data quantiles')
    ax.set_xlabel('Normal theoretical quantiles')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_title(title, fontsize=10)

In [ ]:
import matplotlib.mlab as mlab

norm_resid = (linear_fit_with_surge.resid - linear_fit_with_surge.resid.mean()) / linear_fit_with_surge.resid.std() 

n, bins, patches = plt.hist(norm_resid, density = True)
plt.ylabel('Density')
plt.xlabel('Normalized residuals');

# best fit of data
(mu, sigma) = norm.fit(norm_resid)

# the histogram of the data
#n, bins, patches = plt.hist(datos, 60, normed=1, facecolor='green', alpha=0.75)

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.title(r'$\mathrm{Histogram\ and\ density\ fit\ of\ normalized\ residuals:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
plt.grid(True)
plt.show()

The distribution of the normalized residuals of the full model is close to a normal distribution.

## Outliers
Outliers can have an impact on the parameter estimates and the fitted residuals. Below we first plot the fitted sea levels including confidence and prediction intervals. Next we plot the residuals against time and against leverage.

In [ ]:
# Code om betrouwbaarheidsintervallen en predictie-intervallen te maken

# Confidence intervals paramemeters using t-distribution
# print(linear_fit_with_surge.conf_int(alpha=0.05, cols=None)) 
## linear_fit_with_surge.conf_int_el # using empirical likelihood

from statsmodels.stats.outliers_influence import summary_table, OLSInfluence
lfws = linear_fit_with_surge
st, data, ss2 = summary_table(lfws, alpha=0.05)
#print(lfws.model.nobs)
#print(lfws.model.exog_names)

fittedvalues = data[:,2]
predict_mean_se  = data[:,3]
predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T # confidence interval
predict_ci_low, predict_ci_upp = data[:,6:8].T # prediction interval

# check we got the right things
#print(fittedvalues)
#print(np.max(np.abs(lfws.fittedvalues - fittedvalues)))
#print(np.max(np.abs(iv_l - predict_ci_low))) #iv_l and iv_u are lower and upper values prediction interval from another method
#print(np.max(np.abs(iv_u - predict_ci_upp))) #

In [ ]:
# plot the model with storm surge and add confidence and prediction intervals
# grafieken van de predictie-intervallen van het regressiemodel en de waarnemingen.
for startyear in [1890, 1979]:

    fig = bokeh.plotting.figure(x_range=(startyear, 2020), plot_width=900, plot_height=400)

    fig.circle(
        mean_df.year,
        mean_df.height,
        line_width=1,
        legend='Annual sea level (data)',
        color='black',
        alpha=0.5
    )

    fig.line(
        linear_fit_with_surge.model.exog[:, 1] + 1970, 
        linear_fit_with_surge.predict(), 
        line_width=3, 
        legend='Current sea level (linear model with surge)', 
        color='black',
        alpha=0.5
    )
    
    for values, label, color, lw in zip([predict_ci_low, predict_ci_upp, predict_mean_ci_low, predict_mean_ci_upp],
                             ['95% prediction interval', '', '95% confidence interval', ''],
                             ['blue', 'blue', 'red', 'red'],
                             [2, 2, 1, 1]):
        fig.line(
            linear_fit_with_surge.model.exog[:, 1] + 1970, 
            values, 
            line_width=lw, 
            legend=label, 
            color=color,
            alpha=0.5
        )

    fig.legend.location = "bottom_right"
    fig.yaxis.axis_label = 'water level [mm] above NAP'
    fig.xaxis.axis_label = 'year'
    #fig.legend.click_policy = "hide"

    bokeh.io.show(fig)

Only a few observations are outside the prediction intervals. It is remarkable to see so many observations in the confidence interval. The sea level in year 1996 is comparatively low. The fit is good though.

In [ ]:
# Controle op uitbijters (outlier detection): standaard residuenplot 

# residual plot
fig = bokeh.plotting.figure(x_range=(1890, 2020), plot_width=900, plot_height=400)
fig.circle(
    mean_df.year,
    linear_fit_with_surge.resid,
    line_width=1,
    legend='Residuals',
    color='black',
    alpha=0.5
)

fig.legend.location = "top_left"
fig.yaxis.axis_label = 'water level [mm] above NAP'
fig.xaxis.axis_label = 'year'
#fig.legend.click_policy = "hide"

bokeh.io.show(fig)

print('The graph shows a sort of cyclical pattern in the residuals implying autocorrelation.')

print('From 2005 onwards the residuals increase. The residuals between 2014-2017 are relatively large, but in the early 1960s the residuals have been large as well.') 

print('The lack of storm surge data from 2015 onwards is likely to contribute to the high values in 2015-2017.')

In [ ]:
# influence = linear_fit_with_surge_1979.get_influence()
# frame = influence.summary_frame()
# display(frame.sort_values(by='hat_diag', ascending=False).head())

# print(linear_fit_with_surge.model.exog[106, 1] + 1970)
# fig, ax = plt.subplots()

# ax.scatter(frame['hat_diag'], frame['standard_resid'], s=20, alpha=0.5)
# ax.set_ylabel('Standardized residuals')
# ax.set_xlabel('Leverage');

fig, axs = plt.subplots(figsize=(14, 8), ncols=2)
plots.plot_leverage_resid2(linear_fit_with_surge, ax=axs[0]);
plots.plot_leverage_resid2(linear_fit_with_surge_1979, ax=axs[1]);

for ax in axs:
    ax.set_ylim(0.0, 0.35)
    ax.set_xlim(0.0, 10.0)
    ax.grid()

In [ ]:
fig, axs = plt.subplots(figsize=(14, 8), ncols=2)
plots.influence_plot(linear_fit_with_surge, ax=axs[0]);
plots.influence_plot(linear_fit_with_surge_1979, ax=axs[1]);

for ax in axs:
    ax.set_ylim(-2.5, 2.5)
    ax.set_xlim(0.0, 0.35)
    ax.grid()

The above plots show that the year 1996 has the highest leverage. The influence is relatively high, but some other years with the same leverage have the same influence. 

### Serial correlation: Breusch-Godfrey test
The null hypothesis of the Breusch-Godfrey test is no serial correlation.

#### References
Breusch, T. S. (1978). "Testing for Autocorrelation in Dynamic Linear Models". Australian Economic Papers. 17: 334–355. doi:10.1111/j.1467-8454.1978.tb00635.x.

Godfrey, L. G. (1978). "Testing Against General Autoregressive and Moving Average Error Models when the Regressors Include Lagged Dependent Variables". Econometrica. 46: 1293–1301. JSTOR 1913829.

In [ ]:
import pandas as pd
import statsmodels

fig = bokeh.plotting.figure(x_range=(1890, 2020), y_range=(-80, 80), plot_width=900, plot_height=400)

resid_df = pd.DataFrame(
    data=linear_fit_with_surge.resid.values,
    index=mean_df_corrected.year,
    columns=['residuals']
)
resid_df['5 year rolling average'] = resid_df.rolling(window=5, center=True).mean()

fig.line(resid_df.index.values, resid_df['residuals'].values,
         line_width=2, color='blue', alpha=0.5, legend='Residuals')
fig.line(resid_df.index.values, resid_df['5 year rolling average'].values,
         line_width=2, color='red', alpha=0.5, legend='Moving average (5 years)')


fig.legend.location = "bottom_right"
fig.yaxis.axis_label = 'water level [mm] above NAP'
fig.xaxis.axis_label = 'year'

bokeh.io.show(fig)

# statsmodels.stats.diagnostic.acorr_breusch_godfrey?
bg = statsmodels.stats.diagnostic.acorr_breusch_godfrey(linear_fit_with_surge)

print("""
Lagrange multiplier test statistic: {:.3f}
P-value for Lagrange multiplier test: {:.3f}
Fstatistic for F test: {:.3f}
P-value for F test: {:.3f}
""".format(*bg))

The null hypothesis "No serial correlation" is rejected at the 5% significance level. This is in line with the outcome of the Durbin-Watson test (on autocorrelation). This conclusion means that the covariance matrix and hence the standard errors might be too small.

One way of dealing with autocorrelation is estimating the standard errors in a more robust way (White). 

Another way of dealing with autocorrelation is using an AR(p) model. Therefore, we shall first check the autocorrelation function to see what the lag is in the dependent variable.

Remark. In case of autocorrelation and heteroscedasticity one can apply the HAC estimator to retrieve robust standard errors (statsmodels.stats.sandwich_covariance.cov_hac; Newey-West covariance matrix estimator). In the last section of this notebook we  test on homoscedasticity and implement robust standard errors.

In [ ]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[np.int(result.size/2):]

plt.plot(autocorr(resid_df['residuals'].values))

#from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Remark: 
# ACF from residuals and mean sea level
plot_acf(resid_df['residuals'].values, lags=10)
pyplot.show()

plot_acf(mean_df.height.values, lags=10)
pyplot.show()

# PACF from residuals and mean sea level
plot_pacf(resid_df['residuals'].values, lags=10)
pyplot.show()

plot_pacf(mean_df.height.values, lags=10)
pyplot.show()

The ACF and partial ACF of the residuals and the sea levels show autoregression with lag 1. This is in line with the outcomes of the statistical tests on autocorrelation and serial correlation. We could estimate an AR(1) model to deal with the autocorrelation. Neglecting autocorrelation could lead to incorrect standard errors. Another option is 'correcting' the standard errors for autocorrelation (and other types misspecification - heteroscedasticity).

### Homoscedastic or heteroskedastic residuals: 
#### Breusch-Pagan test
The null hypothesis of the Breusch-Pagan and White test is homoscedastic residuals. The White test is a special case of the B-P test.

Breusch, T. S.; Pagan, A. R. (1979). "A Simple Test for Heteroskedasticity and Random Coefficient Variation". Econometrica. 47 (5): 1287–1294. doi:10.2307/1911963. JSTOR 1911963. MR 0545960.

Cook, R. D.; Weisberg, S. (1983). "Diagnostics for Heteroskedasticity in Regression". Biometrika. 70 (1): 1–10. doi:10.1093/biomet/70.1.1.

White, H. (1980). "A Heteroskedasticity-Consistent Covariance Matrix Estimator and a Direct Test for Heteroskedasticity". Econometrica. 48 (4): 817–838. JSTOR 1912934. MR 0575027.

First we plot the residuals against the main independent variables: nodal cycle and surge. Subsequently we perform the B-P test and the White test.

In [ ]:
def predict_nodal(C, D, year):
    """
    C : float
        Coefficient from the sin-component
    D : float
        Coefficient from the cos-component
    """
    
    # Recreate the nodal cycle from the sinus and cosinus components.
    # It does not matter that we use the time-independent parameters.
    phi = np.arctan(D / C)
    A = (C**2 + D**2) ** 0.5
    Tn = 18.613
    
    return A * np.sin((2 * np.pi * year) / Tn + phi)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(14, 10))

for i, (yearmin, fit, title) in enumerate(zip(
    [1890, 1979, 1979],
    [linear_fit_with_surge, linear_fit_with_surge, linear_fit_with_surge_1979],
    ['All residuals from fit 1890-2017', 'Residuals 1979-2017 from fit 1890-2017', 'All residuals from fit 1979-2017']
)):

    # Select years
    indices = mean_df['year'] >= yearmin

    # Get surge and residuals for fit and years
    surge = mean_df.loc[indices, 'surge'].values
    residuals = fit.resid.tail(len(surge)).values
    
    # Plot surge
    ax = axs[0, i]
    ax.scatter(surge, residuals, s=10)
    ax.set_xlabel('Surge [mm]')
    ax.set_xlim(-75, 75)
    ax.set_aspect(1.0)
    
    # Format
    for ax in axs[:, i]:
        ax.grid()
        ax.set_ylabel('Residuen [mm]')
        ax.set_ylim(-75, 75)    
    
    # Nodal
    C, D = linear_fit_with_surge.params.x2, linear_fit_with_surge.params.x3
    nodal_pred = predict_nodal(C, D, mean_df.loc[indices, 'year'])
    ax = axs[1, i]
    ax.scatter(nodal_pred, residuals, s=10)
    ax.set_xlabel('Nodal cyclus [mm]')
    ax.set_xlim(-15, 15)
    ax.set_aspect(0.2)

    # Breusch-Pagan test
    bp = statsmodels.stats.diagnostic.het_breuschpagan(residuals, mean_df.loc[indices].values)

    print("""
    {}:
    ---------------------------------------
    Lagrange multiplier test statistic: {:.3f}
    P-value for Lagrange multiplier test: {:.3f}
    Fstatistic for F test: {:.3f}
    P-value for F test: {:.3f}
    """.format(title, *bp))
    
    # https://www.statsmodels.org/dev/stats.html   


The plots show that the spread in the residuals somewhat depends on the value of the nodal cycle and the surge. Therefore it is not suprising that the null hypothesis of the Breusch-Pagan test is rejected at the 5% significance level.

In [ ]:
# Remark. The White test and Goldfeld-Quandt test are not yet applicable in Python's statsmodels module. 

    # White test
    wt = statsmodels.stats.diagnostic.het_white(residuals, mean_df.loc[indices].values)

    print("""
    {}:
    ---------------------------------------
    Lagrange multiplier test statistic: {:.3f}
    P-value for Lagrange multiplier test: {:.3f}
    Fstatistic for F test: {:.3f}
    P-value for F test: {:.3f}
    """.format(title, *wt))
    
    # Goldfeld-Quandt test
    gq = statsmodels.stats.diagnostic.het_goldfeldquandt(residuals, mean_df.loc[indices].values)

    print("""
    {}:
    ---------------------------------------
    Lagrange multiplier test statistic: {:.3f}
    P-value for Lagrange multiplier test: {:.3f}
    Fstatistic for F test: {:.3f}
    P-value for F test: {:.3f}
    """.format(title, *gq))

#### Residuals vs. fitted values
The same conclusions are drawn after plotting the residuals against the fitted sea water levels. The spread is higher for the lower fitted sea levels. This is because the lower fitted values relate to the begin period and in that period (until 1979) the surge does not explain any variance. Adding the surge to the model from the year 1979 onwards yields lower variation in the residuals. Still, the residuals show more variation for the lower and higher fitted values.

This leads to the final conclusion that the model describes the data well and that most model assumptions are reasonable. There is some evidence of serial corrrelation and heteroscedasticity in the residuals. Therefore the model requires robust estimation of standard errors. This can be easily incorporated with the option "cov_type='HC0' or "cov_type='hac'" in the function "model.fit()". For example, "fit = model.fit(cov_type='HC0')". The two covariance estimators are applied below.

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(13, 6))

for i, (yearmin, fit, title) in enumerate(zip(
    [1890, 1979, 1979],
    [linear_fit_with_surge, linear_fit_with_surge, linear_fit_with_surge_1979],
    ['All residuals from fit 1890-2017', 'Residuals 1979-2017 from fit 1890-2017', 'All residuals from fit 1979-2017']
)):

    # Select years
    indices = mean_df['year'] >= yearmin

    # Get surge and residuals for fit and years
    predicted = fit.fittedvalues[indices]
    residuals = fit.resid.tail(len(predicted)).values
    
    # Plot surge
    ax = axs.ravel()[i]
    ax.scatter(predicted, residuals, s=10)
    ax.set_xlabel('Fitted [mm + NAP]')
    ax.set_xlim(-200, 200)
    ax.set_aspect(1.0)
    
    ax.grid()
    ax.set_ylabel('Residual [mm]')
    ax.set_ylim(-100, 100)   
    
    ax.set_title(title)
    
plt.tight_layout()

In [ ]:
def linear_model_with_surge_robust_hc0(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613),
        df['surge']# * (df['year'] >= 1979)    
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']
        
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    #fit = model.fit()
    fit = model.fit(cov_type='HC0')
    return fit, names

In [ ]:
display(table) # summary table regression results with 'uncorrected' standard errors

linear_fit_with_surge, names = linear_model_with_surge_robust_hc0(mean_df)
table_hc0 = linear_fit_with_surge.summary(
    yname='Sea-surface height', 
    xname=names, 
    title='Linear model (1890 - current), with surge (1979-current)'
)
display(table_hc0)

In [ ]:
def linear_model_with_surge_robust_hac(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613),
        df['surge']# * (df['year'] >= 1979)    
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']
        
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    fit = model.fit(cov_type='HAC', cov_kwds={'maxlags':1}) # 1 lag (see ACF)
    return fit, names

In [ ]:
linear_fit_with_surge, names = linear_model_with_surge_robust_hac(mean_df)
table_hac = linear_fit_with_surge.summary(
    yname='Sea-surface height', 
    xname=names, 
    title='Linear model (1890 - current), with surge (1979-current)'
)
display(table_hac)